<a href="https://colab.research.google.com/github/yunjichu/image/blob/main/image_project(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 설정
- colab 파일에 첨부된 js파일 옮기기

In [1]:
#yt-dlp 설치
!pip install yt-dlp opencv-python-headless

# 패키지 목록 업데이트
!apt-get update

# FFmpeg 설치
!apt-get install ffmpeg -y

#opencv 설치
!pip install opencv-python-headless

#google cloud vision 설치
!pip install google-cloud-vision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 27.3 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,514 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,

# 유튜브에서 영상 다운 받아 1초 간격으로 프레임 추출

In [2]:
import os
from yt_dlp import YoutubeDL

def download_youtube_video(url, save_path="downloaded_video.mp4"):
    """
    Downloads a YouTube video using yt-dlp.
    """
    ydl_opts = {
        'format': 'bestvideo+bestaudio/best',  # 최고 품질로 다운로드
        'outtmpl': save_path,  # 저장 파일 경로
        'noplaylist': True,    # 재생 목록 제외
    }
    with YoutubeDL(ydl_opts) as ydl:
        print(f"Downloading video from: {url}")
        ydl.download([url])
    print(f"Video downloaded to: {save_path}")

def extract_frames_every_n_seconds(video_path, output_dir, interval_seconds=1):
    import cv2
    """
    Extracts frames from a video file every `interval_seconds` seconds.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"Total Frames: {frame_count}, FPS: {fps}")

    # 매 interval_seconds마다 프레임 추출
    frame_interval = int(fps * interval_seconds)
    print(f"Extracting frames every {interval_seconds} seconds ({frame_interval} frames interval).")

    frame_number = 0
    saved_frames = 0
    success, frame = cap.read()

    if not success:
        print("Error: Unable to read the video file.")
        return

    while success:
        # 지정된 간격에 해당하는 프레임만 저장
        if frame_number % frame_interval == 0:
            frame_filename = os.path.join(output_dir, f"frame_{frame_number:05d}.jpg")
            cv2.imwrite(frame_filename, frame)
            print(f"Saved frame: {frame_filename}")
            saved_frames += 1

        success, frame = cap.read()
        frame_number += 1

    cap.release()
    print(f"Frames extracted every {interval_seconds} seconds saved to: {output_dir}")
    print(f"Total saved frames: {saved_frames}")

# YouTube URL 및 다운로드 경로 설정
youtube_url = "https://www.youtube.com/watch?v=WinIUBhMnFU&t=63s"
video_path = "downloaded_video.mp4"

# 동영상 다운로드 실행
download_youtube_video(youtube_url, save_path=video_path)

# 추가적인 yt-dlp 명령 실행
!yt-dlp -f best "https://www.youtube.com/watch?v=WinIUBhMnFU&t=63s" -o "downloaded_video.mp4"

if os.path.exists("downloaded_video.mp4"):
    print("File downloaded successfully.")
else:
    print("File download failed.")

# ffmpeg 변환 명령 실행
!ffmpeg -i downloaded_video.mp4 -vcodec libx264 fixed_video.mp4

# 3초 간격으로 프레임 추출 실행
video_path = "fixed_video.mp4"  # 변환된 동영상 파일 경로
output_dir = "all_frames"      # 저장 폴더 이름

extract_frames_every_n_seconds(video_path, output_dir, interval_seconds=1)



[youtube] Extracting URL: https://www.youtube.com/watch?v=WinIUBhMnFU&t=63s
[youtube] WinIUBhMnFU: Downloading webpage
[youtube] WinIUBhMnFU: Downloading ios player API JSON
[youtube] WinIUBhMnFU: Downloading mweb player API JSON
[youtube] WinIUBhMnFU: Downloading player b46bb280
[youtube] WinIUBhMnFU: Downloading m3u8 information
[info] WinIUBhMnFU: Downloading 1 format(s): 399+251
[download] Destination: downloaded_video.mp4.f399.mp4
[download] 100% of   29.88MiB in 00:00:01 at 29.03MiB/s  
[download] Destination: downloaded_video.mp4.f251.webm
[download] 100% of    2.95MiB in 00:00:00 at 18.98MiB/s  
[Merger] Merging formats into "downloaded_video.mp4.webm"
Deleting original file downloaded_video.mp4.f251.webm (pass -k to keep)
Deleting original file downloaded_video.mp4.f399.mp4 (pass -k to keep)
Video downloaded to: downloaded_video.mp4
         To let yt-dlp download and merge the best available formats, simply do not pass any format selection.
         If you know what you are d

# 자막 추출
- google-vision-api 사용

In [3]:
import cv2
import os
from google.cloud import vision

# 환경변수 설정
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/image-443101-514bb45e53e7.json"
print("서비스 계정 키 경로 설정 완료.")

# Vision API 클라이언트 초기화
client = vision.ImageAnnotatorClient()

# 디렉토리 경로 설정
all_frame_dir = "/content/all_frames"
output_dir = "/content/output_texts"

# 출력 디렉토리 생성
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 통합 텍스트 파일 경로
output_text_file = os.path.join(output_dir, "extracted_texts.txt")

# 이미지 파일 리스트 가져오기
image_files = [f for f in os.listdir(all_frame_dir) if f.endswith(".jpg")]
image_files.sort()  # 파일 이름 기준 정렬

# 기존 텍스트 파일 초기화
with open(output_text_file, "w") as f:
    f.write("### Vision API 추출 결과 ###\n\n")

# 각 이미지 파일 처리
for image_file in image_files:
    image_path = os.path.join(all_frame_dir, image_file)
    print(f"처리 중: {image_path}")

    try:
        # 이미지 읽기
        image = cv2.imread(image_path)

        # OpenCV로 이미지 인코딩
        success, encoded_image = cv2.imencode(".jpg", image)
        if not success:
            print(f"이미지를 인코딩하지 못했습니다: {image_file}")
            continue

        # Vision API 요청 준비
        content = encoded_image.tobytes()
        image_vision = vision.Image(content=content)
        response = client.text_detection(image=image_vision)

        # 텍스트 추출
        texts = response.text_annotations
        extracted_text = texts[0].description if texts else "텍스트를 감지하지 못했습니다."
        print(f"추출된 텍스트:\n{extracted_text}")

        # 개별 텍스트 파일 저장
        individual_text_file = os.path.join(output_dir, f"{image_file.split('.')[0]}.txt")
        with open(individual_text_file, "w") as f:
            f.write(extracted_text)

        # 통합 텍스트 파일에 추가
        with open(output_text_file, "a") as f:
            f.write(f"### {image_file} ###\n{extracted_text}\n\n")

    except Exception as e:
        print(f"오류 발생: {image_file} - {str(e)}")
        continue

print(f"모든 텍스트를 {output_text_file}에 저장 완료.")

서비스 계정 키 경로 설정 완료.
처리 중: /content/all_frames/frame_00000.jpg
추출된 텍스트:
텍스트를 감지하지 못했습니다.
처리 중: /content/all_frames/frame_00029.jpg
추출된 텍스트:
일요일 출근
처리 중: /content/all_frames/frame_00058.jpg
추출된 텍스트:
일요일 출근
처리 중: /content/all_frames/frame_00087.jpg
추출된 텍스트:
일요일 출근
처리 중: /content/all_frames/frame_00116.jpg
추출된 텍스트:
일요일 출근
처리 중: /content/all_frames/frame_00145.jpg
추출된 텍스트:
텍스트를 감지하지 못했습니다.
처리 중: /content/all_frames/frame_00174.jpg
추출된 텍스트:
???
처리 중: /content/all_frames/frame_00203.jpg
추출된 텍스트:
???
처리 중: /content/all_frames/frame_00232.jpg
추출된 텍스트:
텍스트를 감지하지 못했습니다.
처리 중: /content/all_frames/frame_00261.jpg
추출된 텍스트:
A
처리 중: /content/all_frames/frame_00290.jpg
추출된 텍스트:
야 모라고?
처리 중: /content/all_frames/frame_00319.jpg
추출된 텍스트:
야 모라고?
처리 중: /content/all_frames/frame_00348.jpg
추출된 텍스트:
텍스트를 감지하지 못했습니다.
처리 중: /content/all_frames/frame_00377.jpg
추출된 텍스트:
텍스트를 감지하지 못했습니다.
처리 중: /content/all_frames/frame_00406.jpg
추출된 텍스트:
텍스트를 감지하지 못했습니다.
처리 중: /content/all_frames/frame_00435.jpg
추출된 텍스트:
아싸라비야
처리 중: